# Challenge 6: 自駕車周邊模組的控制與驅動 (計算機組織 / 作業系統概論 / 軟硬體整合)
2020.11.13

## Requirement

- 控制小鴨車完成循跡地圖
- 在作業 6 的部分，我們已經可以透過 Arduino Nano 接收紅外線循跡模組的訊號，並且透過 Serial port 送出小鴨車的控制訊號。現在，我們想透過送出的這些控制訊號，實際控制小鴨車循跡走完地圖，並且在遇到障礙物時，停止前行。

### Step1
- 將紅外線循跡模組裝到小鴨車的車頭部分
    - 總共需要一個 30 mm + 一個螺帽 + 一個墊片的高度
    - ![](https://i.imgur.com/fyX2MoI.png)
- 將車頭朝前，順時針旋轉紅外線循跡模組上的可變電阻（白色的那顆），轉到底
    - 找不到可變電阻，或者是不知道什麼是順時針的同學，找助教八～（助教很有耐心的！）
    - ![](https://i.imgur.com/bOTPuQT.png)

### Step2

- 從外觀檢查紅外線循跡模組的表現
    - 將小鴨車放在黑線紙上，當朝下紅外線模組壓在黑線上時，模組上對應的 LED 燈亮起；當朝下紅外線模組壓在白紙上時，模組上對應的 LED 燈暗掉。
    - 當小鴨車前方有障礙物時（約 5 公分），Near 所對應的 LED 燈暗掉；當小鴨車前方沒有障礙物時，Near 所對應的 LED 燈亮起。
        - 若偵測前方障礙物有問題，試著調整朝前的紅外線收發器角度（兩者需要比平行再微微向內縮一點點），若持續有問題，找助教八～
        - ![](https://i.imgur.com/SdSGz5G.png)

### Step 3

- 在作業 6 的時候，我們使用 `Serial.print("...\n")` 將控制訊號透過 USB 送出，因此只要將 Arduino Nano 連接上小鴨車，就可以在小鴨車上由 Serial port 收到我們送出的控制訊號。
    - 修正：請同學將原本 Ardiono code 最後一行的 `delay(100);` 改為 `delay(10);`，助教感到十分抱歉。記得改完後要再燒錄一次！
- 透過 MobaXterm or Royal 連進小鴨車
- 從 github clone project
    - 執行以下 command line
    ```
    cd
    git clone -b Challenge6 https://github.com/GrassLab/NCTU_CS_ZOO Challenge6
    ```
- 安裝相關套件
    - 執行以下 command line
    ```
    cd ~/Challenge6
    pip3 install -r duckiebot_cs_zoo/requirements.txt
    ```
- 取得讀取 Serial port 的權限
    - 執行以下 command line
    ```
    sudo usermod -a -G dialout ${USER}
    newgrp dialout
    ```


### Step4

- 修改控制小鴨車的程式碼
    - 小鴨車的控制程式碼在 `~/Challenge5/duckiebot_cs_zoo/app/ir_joystick.py`，同學可以透過 MobaXterm or Royal 的編輯器開啟，或是透過 command line 輸入 `vim ~/Challenge5/duckiebot_cs_zoo/app/ir_joystick.py` 進行編輯。
        - 若編輯或存檔有問題，助教幫你！
    - 打開後可以看到以下的程式碼。

```python
import serial
import time
try:
    from libs.car_control import CarControl
except ImportError:
    from ..libs.car_control import CarControl

# use CarControl library to control your car
# you can set up trim_val if your car cannot walk straightly when velocity is 1 and omega is 0
# -0.03 to 0.03 is the suitable range of trim_val, but it actually depends on the motors of your car 
duckie_car = CarControl(trim_val=0.0)

ser = serial.Serial('/dev/ttyUSB0', 9600)
time.sleep(1)
ser.flushInput()

while True:
    # clear the previous message in buffer
    ser.flushInput()
    # get the newest message from the serial port
    data = ser.readline().decode("ascii")
    key = data[:-1]

    # omega range is -8 to 8. The negative number means turning right, and the positive number means turing left.
    # velocity range is -1 to 1. 0 is stop. 1 is moving forward with full speed. 0.3 is a good choice when you want to move forward.
    omega = 0
    velocity = 0
    # change the control message you use in Arduino nano, and give the corresponding omega and velocity
    if key == "move":
        omega = 0
        velocity = 0.3
    elif key == "left":
        omega = 2
        velocity = 0.15
    elif key == "right":
        omega = -2
        velocity = 0.15
    elif key == "stop":
        omega = 0
        velocity = 0
    else:
        omega = 0
        velocity = 0

    # call move with velocity and omega can control your little duck car
    duckie_car.move(
        velocity=velocity,
        omega=omega,
    )
```

    - 請研究如何使用 `CarControl` 對小鴨車的車輪進行控制，並修改其中的判斷式，使小鴨車能夠順利地在循跡地圖上沿黑線行走。
        - 如同註解中所描述的，`velocity` 的範圍為 -1 到 1 的浮點數，而 `omega` 則是 -8 到 8 的浮點數（正值左轉，負值右轉）
    - 執行以下 command line 以運行程式碼
        ```
        cd ~/Challenge6/duckiebot_cs_zoo
        python -m app.ir_joystick
        ```
        - 用兩週了，應該知道上面的指令在做什麼八～助教深深的相信你們都知道 =))，如果不知道，記得問助教。

## Notice

- Reward
    - 根據你的小鴨車在地圖上行走的距離，決定本週牌位！
    - 測試過程若小壓車車輪卡住，允許推他一把『一次』！！！因此請在小鴨車的車速上做一個正確的抉擇 =))
    - 每個人的小鴨車都要通過考驗！！！！！

## Map

- ![](https://i.imgur.com/l21dw06.png)